# <span style="color:purple">GIS and Machine Learning for Object Detection in Satellite Imagery</span>

<img src="img/robot.jpg"></img>


## <span style="color:blue">Step 4: Generate TF Records from the train/test splits</span>

To create TFRecord files that we need to train an object detection model in TensorFlow.